In [ ]:
import sys
sys.path.append("..")

import pandas as pd
import numpy as np
from pymongo import MongoClient
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from multiprocessing import Process, Pool
import matplotlib.pyplot as plt

from data_transformation.db_env import DbEnv, db
from data_preprocessing.data_tidying import DataTidying




conn, cursor = DbEnv().connect_sql()


def get_youtube_index(num):
    sql = 'SELECT distinct video_num from mu_tech.daily_youtube where song_num=%s' % (num)
    cursor.execute(sql)
    conn.commit()
    list_temp = cursor.fetchall()
    list_num = [x[0] for x in list_temp]

    pool = Pool(20)

    df_youtube = pd.DataFrame()
    df_price_temp = pool.map(get_df_youtube, list_num)
    for i in df_price_temp:
        df_youtube = pd.concat([df_youtube, i], axis=1)
    df_youtube = df_youtube.T

    return df_youtube


def get_df_youtube(num):
    sql = 'SELECT viewCount, date from mu_tech.daily_youtube where video_num=%s ORDER by date' % (num)
    df_temp = db(cursor, sql).dataframe
    df_temp = df_temp.set_index('date')
    df_temp.columns = [num]

    return df_temp

def get_df_price(num):
    sql = 'SELECT price_close, date from mu_tech.musiccowdata where num=%s ORDER by date' % (num)
    df_temp = db(cursor, sql).dataframe
    df_temp = df_temp.set_index('date')
    df_temp.columns = [num]

    return df_temp

def get_average_index(df_youtube):
    df_sa = df_youtube.mean()
    df_lwa_temp = 1 + df_youtube.diff(axis=1).iloc[:, 1:].div(df_youtube.iloc[:, 1:]).mean()
    df_lwa_temp[df_youtube.columns[0]] = 1
    df_lwa_temp = df_lwa_temp.sort_index()
    df_lwa_first = df_youtube.iloc[:, 0].mean(axis=0)
    df_lwa = pd.Series()
    df_lwa[df_youtube.columns[0]] = df_lwa_first

    for idx, row in enumerate(df_lwa_temp[1:].items()):
        df_lwa[row[0]] = row[1] * df_lwa[idx]

    df_sa, df_lwa = df_sa.to_frame(), df_lwa.to_frame()

    return df_sa, df_lwa



In [ ]:
num = 26
df_youtube = get_youtube_index(num)
df_sa, df_lwa = get_average_index(df_youtube)
df_price = get_df_price(num)
df_price = df_price.loc[df_sa.index.tolist(), :]

se_price, se_sa, se_lwa = df_price.squeeze(), df_sa.squeeze(), df_lwa.squeeze()

fig, axs = plt.subplots(3)
axs[0].plot(se_price)
axs[1].plot(se_sa)
axs[2].plot(se_lwa)

# plt.show()